In [ ]:
from typing import TypedDict,Annotated,Sequence
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,AIMessage,BaseMessage,ToolMessage,SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,END
from langgraph.prebuilt import ToolNode

load_dotenv()

#Defining the global variable to store document content
document_content = ""


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


@tool
def update(content: str) -> str:
    """Update the document content."""
    global document_content
    document_content = content
    return f"Document updated. \n{document_content}"


@tool
def save(filename: str) -> str:
    """Save the document content to a file and finish the process.
        args:
            filename: The name of the file to save the document content.
    """
    global document_content
    
    if not filename.endswith(".txt"):
        filename =f"{filename}.txt"
    
    try:
        with open(filename, "w") as file:
            file.write(document_content)
        return f"Document saved to {filename}. Process finished."
    except Exception as e:
        return f"Failed to save document: {str(e)}"


tools=[update,save]

model=ChatOpenAI(model='gpt-4o').bind_tools(tools)

def our_agent(state:AgentState)->AgentState:
    system_prompt=SystemMessage(
        content=f"""You are a helpful assistant that helps users draft documents. You have access to two tools: 'update' to update the document content and 'save' to save the document to a file and finish the process. Always use 'save' when the user indicates they are done draftinng.
        -If user wants to add or modify content, use the 'update' tool.
        -If user wants to finish the process, use the 'save' tool with the desired
        -make sure to always show the current document state after modification

        The current document content is:{document_content}
        """)
    if not state['messages']:
        user_input="I'm ready to help you update a document. What would you like to do?"
        user_message=HumanMessage(content=user_input)
    else:
        user_input=input("\n What would you like to do with document? ")
        print(f"\n User Input: {user_input}")
        user_message=HumanMessage(content=user_input)
    
    all_messages=[system_prompt] + list(state['messages']) + [user_message]

    response=model.invoke(all_messages)

    print(f"\n Agent Response: {response.content}")
    if hasattr(response,'tool_calls') and response.tool_calls:
        print(f"\n Using tools: {[tc['name'] for tc in response.tool_calls]}")
    
    return {"messages":list(state["messages"]) + [user_message,response]}

def should_continue(state:AgentState)->AgentState:
    """Determine whether to continue or end the process"""
    messages=state['messages']

    if not messages:
        return "continue"
    
    #This looks for the most recent tool message
    for message in reversed(messages):
        if (isinstance(message,ToolMessage) and "saved" in message.content.lower() and "document" in message.content.lower()):
            return "end"

    return "continue" 

def print_message(messages):
    """Function i made to print messages in  a more readable format"""
    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message,ToolMessage):
            print(f"\n Tool Result: {message.content}")


graph = StateGraph(AgentState)

graph.add_node("agent",our_agent)
graph.add_node("tools",ToolNode(tools))

graph.set_entry_point("agent")
graph.add_edge("agent","tools")

graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue":"agent",
        "end":END
    }
)

app=graph.compile()

def run_document_agent():
    print("\n ====== DRAFTER ======")

    state={"messages":[]}

    for step in app.stream(state,stream_mode="values"):
        if "messages" in step:
            print_message(step['messages'])

    print("\n ====== DRAFTER FINISHING ======")


if __name__ == "__main__":
    run_document_agent()
